In [ ]:
import glob
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np
import cv2 as cv
import json
import os
import pandas as pd
from tqdm import tqdm
import operator

import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['axes.grid'] = False
mpl.rcParams['figure.figsize'] = (12,12)
import matplotlib.image as mpimg

import geojson
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from Utils.Postprocessing.post_process import postprocess_mask, merge_nearest_endocards

In [ ]:
def IoU(y_true, y_pred):
    y_true = y_true.flatten()
    y_pred = y_pred.flatten()
    
    intersection = np.logical_and(y_true, y_pred)
    union = np.logical_or(y_true, y_pred)
    iou_score = np.sum(intersection) / np.sum(union)
    return iou_score

In [ ]:
def dice_coef(y_true, y_pred):
    y_true = y_true.flatten()
    y_pred = y_pred.flatten()
    
    intersect = np.sum(y_true * y_pred)
    fsum = np.sum(y_true)
    ssum = np.sum(y_pred)
    dice = (2 * intersect ) / (fsum + ssum)
    return dice

In [ ]:
def convert_to_multiclass(mask, num_classes):
    new_mask = np.zeros((mask.shape[0], mask.shape[1], num_classes), dtype=np.uint8)
    
    new_mask[:, :, 4] = mask[:, :, 0]
    new_mask[:, :, 5] = mask[:, :, 1]
    new_mask[:, :, 6] = mask[:, :, 2]
    
    new_mask[:, :, 1] = cv.bitwise_and(new_mask[:, :, 4], new_mask[:, :, 5])    # blood + infla
    new_mask[:, :, 2] = cv.bitwise_and(new_mask[:, :, 6], new_mask[:, :, 5])    # endocard + infla
    new_mask[:, :, 3] = cv.bitwise_and(new_mask[:, :, 4], new_mask[:, :, 6])    # blood + endocard
    
    union = new_mask[:, :, 0]
    for i in range(1, num_classes):
        union = cv.bitwise_or(union, new_mask[:, :, i])
    union = np.clip(union, 0, 1)
    new_mask[:, :, 0] = np.where((union == 0) | (union == 1), union ^ 1, union)

    return new_mask

In [ ]:
def get_mask_index(feature, classes):
    class_type = feature['properties']['classification']['name']

    for idx, name in enumerate(classes):
        if class_type.lower() == name.lower():
            return idx

    # else return Other cells
    return 0


def get_mask(shape, annotations, classes):
    x, y = int(shape[0]), int(shape[1])

    classes_masks = [
        np.zeros((x, y, 1), dtype='uint8')
        for _ in range(len(classes))
    ]

    for feat in annotations:
        geometry_name = 'geometry'
        coors = feat[geometry_name]['coordinates'][0]
        try:
            pts = [[round(c[0]), round(c[1])] for c in coors]
        except:
            pts = [[round(c[0]), round(c[1])] for c in coors[0]]
        cv.fillPoly(
            classes_masks[get_mask_index(feat, classes)],
            [np.array(pts)],
            1
        )  # fill with ones if cell present

    mask = np.concatenate(classes_masks, axis=2)
    return mask

In [ ]:
def vizualize_cm(cm, num_classes=7):
    classes = [_ for _ in range(num_classes)]
    classes_names= ['Pozadie', 'Cieva + zápal', 'Endokard + zápal', 'Cieva + endokard', 'Cieva', 'Zápal', 'Endokard']
    
    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    cm = np.nan_to_num(cm)
    fig, ax = plt.subplots(figsize=(10, 10))
    im = ax.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes_names, yticklabels=classes_names,
           #title='Normalized Confusion Matrix',
           ylabel='Skutočná trieda',
           xlabel='Predikovaná trieda')

    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
         rotation_mode="anchor")
    
    fmt = '.2f' #'d' # if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout(pad=2.0, h_pad=2.0, w_pad=2.0)
    ax.set_ylim(len(classes)-0.5, -0.5)
    
    plt.savefig("StackedUNet LAB inference cm.svg")

In [ ]:
def get_metrics(df, cm_values, ground_truth_path, predicted_path, classes, shape, name, num_classes=7):
    data = {
        'name': name
    }
    gj = geojson.load(open(ground_truth_path))
    ground_truth = get_mask(shape, gj['features'], classes)
    
    gj = geojson.load(open(predicted_path))
    predicted = get_mask(shape, gj['features'], classes)
    #predicted = postprocess_mask(predicted, True)
    predicted = merge_nearest_endocards(predicted)
    
    for c_idx, c in enumerate(classes):
        data[f'IoU {c}'] = IoU(ground_truth[:, :, c_idx], predicted[:, :, c_idx])
        data[f'Dice {c}'] = dice_coef(ground_truth[:, :, c_idx], predicted[:, :, c_idx])

    ground_truth = convert_to_multiclass(ground_truth, num_classes)
    ground_truth = np.argmax(ground_truth, axis=-1)
    predicted = convert_to_multiclass(predicted, num_classes)
    predicted = np.argmax(predicted, axis=-1)
    
    cm = confusion_matrix(ground_truth.flatten().astype('uint8'), predicted.flatten().astype('uint8'), labels=[0, 1, 2, 3, 4, 5, 6])
    cm_values = cm_values + cm
    
    return pd.concat([
            df,
            pd.DataFrame([data])
        ]), cm_values

In [ ]:
def calculate_metrics():
    df = pd.DataFrame()
    cm_values = np.zeros((7, 7))
    
    ground_truth_path = r'D:\Master Thesis\Data\EMB-IKEM-2023-03-16\QuPath project\structure annotations'
    predicted_path = r'D:\IKEM Pracovná cesta\DeepLabV3+ experiments\DeepLabV3+ normal'
    classes = ["Blood vessels", "Inflammation", "Endocarium"]
    
    json_path = r'D:\Master Thesis\Code\Segmentation\data6\images\images_512_256.json'
    
    with open(json_path) as json_file:
        json_data = json.load(json_file)
    
    ground_truth_files = glob.glob(f'{ground_truth_path}\\*.geojson')
    predicted_files = glob.glob(f'{predicted_path}\\*.geojson')
        
    for image in tqdm(json_data['images'], total=len(json_data['images'])):
        name = image['name']
        print(name)
        predicted = [geo for geo in predicted_files if name in geo]
        predicted = predicted[0]
        
        ground_truth = [geo for geo in ground_truth_files if name in geo]
        if len(ground_truth) == 0:
            continue
            
        ground_truth = ground_truth[0]
        shape = (image['height'], image['width'])
    
        df, cm_values = get_metrics(df, cm_values, ground_truth, predicted, classes, shape, name)
    
    return df, cm_values

In [ ]:
df, cm_values = calculate_metrics()

In [ ]:
cm_values

In [ ]:
vizualize_cm(cm_values, 7)

In [ ]:
df

In [ ]:
df.mean()

In [ ]:
metrics_df = pd.DataFrame([
    {
        'Trieda': 'Cievy',
        'IoU [%]': df[df['IoU Blood vessels'] != 0]['IoU Blood vessels'].mean() * 100,
        'Min IoU [%]': df[df['IoU Blood vessels'] != 0]['IoU Blood vessels'].min() * 100,
        'Max IoU [%]': df[df['IoU Blood vessels'] != 0]['IoU Blood vessels'].max() * 100,
        'Dice [%]': df[df['Dice Blood vessels'] != 0]['Dice Blood vessels'].mean() * 100,
        'Min Dice [%]': df[df['Dice Blood vessels'] != 0]['Dice Blood vessels'].min() * 100,
        'Max Dice [%]': df[df['Dice Blood vessels'] != 0]['Dice Blood vessels'].max() * 100,
    },
    {
        'Trieda': 'Zápal',
        'IoU [%]': df[df['IoU Inflammation'] != 0]['IoU Inflammation'].mean() * 100,
        'Min IoU [%]': df[df['IoU Inflammation'] != 0]['IoU Inflammation'].min() * 100,
        'Max IoU [%]': df[df['IoU Inflammation'] != 0]['IoU Inflammation'].max() * 100,
        'Dice [%]': df[df['Dice Inflammation'] != 0]['Dice Inflammation'].mean() * 100,
        'Min Dice [%]': df[df['Dice Inflammation'] != 0]['Dice Inflammation'].min() * 100,
        'Max Dice [%]': df[df['Dice Inflammation'] != 0]['Dice Inflammation'].max() * 100,
    },
    {
        'Trieda': 'Endokard',
        'IoU [%]': df[df['IoU Endocarium'] != 0]['IoU Endocarium'].mean() * 100,
        'Min IoU [%]': df[df['IoU Endocarium'] != 0]['IoU Endocarium'].min() * 100,
        'Max IoU [%]': df[df['IoU Endocarium'] != 0]['IoU Endocarium'].max() * 100,
        'Dice [%]': df[df['Dice Endocarium'] != 0]['Dice Endocarium'].mean() * 100,
        'Min Dice [%]': df[df['Dice Endocarium'] != 0]['Dice Endocarium'].min() * 100,
        'Max Dice [%]': df[df['Dice Endocarium'] != 0]['Dice Endocarium'].max() * 100,
    },
])

In [ ]:
print(metrics_df.round(2).to_latex(index=False)) 